In [145]:
import folium
import json
import random
import ast
import pandas as pd
import branca.colormap as cm

In [146]:
with open('papdata.json', 'r') as f:
    papdata = json.load(f)

with open('patterns.json', 'r') as f:
    patterns = json.load(f)

infected = {}
with open('simulator_results.txt', 'r') as f:
    for line in f:
        if line[0] == '=':
            cur_timestep = line[16:line.index(' ', 16)]
            infected[cur_timestep] = {'delta': [], 'omicron': []}
        elif line[0] == 'd':
            infected[cur_timestep]['delta'] = ast.literal_eval(line[7:])
        elif line[0] == 'o':
            infected[cur_timestep]['omicron'] = ast.literal_eval(line[9:])

df = pd.DataFrame.from_dict(papdata['places'])    
map = folium.Map(location=(df.loc['latitude'].mean(), df.loc['longitude'].mean()))

random.seed(100)

houselocs = {}
for id in papdata['homes'].keys():
    houselocs[id] = {}
    houselocs[id]['latitude'] = random.uniform(df.loc['latitude'].min(), df.loc['latitude'].max())
    houselocs[id]['longitude'] = random.uniform(df.loc['longitude'].min(), df.loc['longitude'].max())

map

In [147]:
for id, data in papdata['places'].items():
    #folium.Marker((data['latitude'], data['longitude']), popup=f"{id}: {data['label']}", icon=folium.Icon()).add_to(map)
    pass
    
map

In [148]:
redcm = cm.LinearColormap(colors=['red','orange'],index=[0,1],vmin=0,vmax=1)
blucm = cm.LinearColormap(colors=['blue','purple'],index=[0,2],vmin=0,vmax=2)

for timestep, locations in patterns.items():
    if int(timestep) < 99945:
        continue
    
    for id, people in locations['places'].items():
        data = papdata['places'][id]
        coords = (data['latitude'], data['longitude'])
        num_infected = len(list(set(people).intersection(infected[timestep]['delta'] + infected[timestep]['omicron'])))
        #folium.Marker(coords, popup=f"{id}: {data['label']}, {len(people)} people", icon=folium.Icon(color='blue', icon='building', prefix='fa')).add_to(map)
        folium.Circle(coords, radius=(len(people) + 10) * 5, popup=f"{id}: {data['label']}, {len(people)} people, {num_infected} infected", color=blucm(num_infected), fill_color=blucm(num_infected), weight=1).add_to(map)
    
    for id, people in locations['homes'].items():
        coords = (houselocs[id]['latitude'], houselocs[id]['longitude'])
        num_infected = len(list(set(people).intersection(infected[timestep]['delta'] + infected[timestep]['omicron'])))
        folium.Circle(coords, radius=(len(people) + 10)*5, popup=f"{id}: {len(people)} people, {num_infected} infected", color=redcm(num_infected), fill_color=redcm(num_infected), weight=1).add_to(map)
        #folium.Marker(coords, popup=f"{id}: {len(people)} people", icon=folium.Icon(color='red', icon='home', prefix='fa')).add_to(map)

    
    display(map)
